In [3]:
import os
import pandas as pd
import json
from glob import glob

tracklet_annotations = "/playpen-storage/levlevi/player-re-id/src/data/50_tracklet_re_id_benchmark/annotations.json"
roster_annotations_df_fp = '/playpen-storage/levlevi/player-re-id/src/data/team_rosters_df.csv'
with open(tracklet_annotations, 'r') as f:
    tracklet_annotations = json.load(f)
roster_annotations_df = pd.read_csv(roster_annotations_df_fp)

In [4]:
ROOT_DIR = '/playpen-storage/levlevi/player-re-id/src/data/50_tracklet_re_id_benchmark/labeled-tracks'

tracks_file_paths = []
team_ids_arr = []
team_colors_arr = []
player_races = []
candidate_team_names = []

for fp, game_dict in tracklet_annotations.items():
    for track_id, tracklet in tracklet_annotations[fp]['tracks'].items():
        player_id = tracklet.get('human_annotation')
        if player_id == None:
            team_id = None
            player_race = None
            team_colors = None
        else:
            player_features = roster_annotations_df[roster_annotations_df['player_id'] == player_id]
            team_id = player_features['team_id'].values[0]
            team_colors = player_features['team_colors'].values[0]
            player_race = player_features['race'].values[0]
        subdir_fp = os.path.join(ROOT_DIR, fp, track_id)
        
        tracks_file_paths.append(subdir_fp)
        team_ids_arr.append(team_id)
        player_races.append(player_race)
        team_colors_arr.append(team_colors)
        candidate_team_names.append(tracklet_annotations[fp]['teams'])

In [5]:
annotations = pd.DataFrame()
annotations['file_path'] = tracks_file_paths
annotations['team_id'] = team_ids_arr
annotations['player_race'] = player_races
annotations['team_colors'] = team_colors_arr
annotations['candidate_team_names'] = candidate_team_names
annotations = annotations.dropna()
annotations.head()

,file_path,team_id,player_race,team_colors,candidate_team_names
1,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,mixed,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']","[milwaukee_bucks, los_angeles_clippers]"
2,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,black,"['#006847', '#F0EAD6', '#0046AD', '#000000']","[milwaukee_bucks, los_angeles_clippers]"
3,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,black,"['#006847', '#F0EAD6', '#0046AD', '#000000']","[milwaukee_bucks, los_angeles_clippers]"
4,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,white,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']","[milwaukee_bucks, los_angeles_clippers]"
5,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,white,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']","[milwaukee_bucks, los_angeles_clippers]"


In [6]:
imgs_df = pd.DataFrame()

rows = []
for row in annotations.iterrows():
    file_path = row[1]['file_path']
    img_file_paths = glob(file_path + '/*.jpg')
    for img_fp in img_file_paths:
        rows.append([img_fp, row[1]['team_id'], row[1]['player_race'], row[1]['team_colors']])

imgs_df = pd.DataFrame(rows, columns=['file_path', 'team_id', 'player_race', 'team_colors'])
imgs_df.head()

,file_path,team_id,player_race,team_colors
0,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,mixed,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']"
1,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,mixed,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']"
2,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,mixed,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']"
3,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,mixed,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']"
4,/playpen-storage/levlevi/player-re-id/src/data...,1.610613e+09,mixed,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']"


In [31]:
# take a random sample of 100 rows from imgs_df
imgs_df = imgs_df.sample(n=100)
# copy all imgs to dir listed below
new_dir = '/playpen-storage/levlevi/player-re-id/src/testing/race_and_team_id_comparisons/100-img-race-team-id-benchmark'

new_file_dst_paths = []
for i, row in imgs_df.iterrows():
    # print(row.file_path)
    os.system(f'cp "{row.file_path}" {new_dir}')
    dst_fp = os.path.join(new_dir, os.path.basename(row.file_path))
    new_file_dst_paths.append(dst_fp)

In [32]:
imgs_df['file_path'] = new_file_dst_paths

In [34]:
imgs_df.to_csv('/playpen-storage/levlevi/player-re-id/src/testing/race_and_team_id_comparisons/100-img-race-team-id-benchmark.csv', index=False)

In [35]:
df = pd.read_csv('/playpen-storage/levlevi/player-re-id/src/testing/race_and_team_id_comparisons/100-img-race-team-id-benchmark.csv')
df

,file_path,team_id,player_race,team_colors
0,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,black,"['#800080', '#FFA500', '#000000', '#808080']"
1,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,black,"['#006847', '#F0EAD6', '#0046AD', '#000000']"
2,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,black,"['#800080', '#FFA500', '#000000', '#808080']"
3,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,mixed,"['#FF0000', '#0000FF', '#C0C0C0', '#000000']"
4,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,black,"['#003DA5', '#FF0000', '#C0C0C0', '#000080']"
...,...,...,...,...
95,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,black,"['#800080', '#FFA500', '#000000', '#808080']"
96,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,black,"['#800080', '#FFA500', '#000000', '#808080']"
97,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,black,"['#006847', '#F0EAD6', '#0046AD', '#000000']"
98,/playpen-storage/levlevi/player-re-id/src/test...,1.610613e+09,white,"['#800080', '#FFA500', '#000000', '#808080']"
